In [ ]:
import os
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime
from itertools import product

import numpy as np
import pandas as pd
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import average_precision_score
from sklearn.model_selection import GroupKFold, GroupShuffleSplit


def train_and_evaluate_final_model(
    X,
    y,
    groups,
    test_size=0.2,
    n_splits=5,
    run_cross_validation=True,
    **hyperparameters,
):
    """Implementation of the final training strategy."""
    # Create held-out test set separating EFO/Gene pairs between train/test
    train_test_split = GroupShuffleSplit(
        n_splits=1, test_size=test_size, random_state=42
    )
    for train_idx, test_idx in train_test_split.split(X, y, groups):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        groups_train = groups[train_idx]

    # Cross-validation
    if run_cross_validation:
        cv_scores = []
        gkf = GroupKFold(n_splits=n_splits)
        print("Performing cross-validation...")
        for fold, (train_idx, val_idx) in enumerate(
            gkf.split(X_train, y_train, groups_train)
        ):
            # Split data
            X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
            y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

            model = GradientBoostingClassifier(
                **hyperparameters, random_state=42, loss="log_loss"
            )
            model.fit(X_fold_train, y_fold_train)
            y_pred_proba = model.predict_proba(X_fold_val)[:, 1]
            avg_precision = average_precision_score(y_fold_val, y_pred_proba)
            cv_scores.append(avg_precision)

            print(f"Fold {fold + 1}: Average Precision = {avg_precision:.3f}")

        print(
            f"\nCross-validation Average Precision: {np.mean(cv_scores):.3f} (+/- {np.std(cv_scores) * 2:.3f})"
        )

    # Train final model on full training set
    print("\nTraining final model on full training set...")
    final_model = GradientBoostingClassifier(
        **hyperparameters, random_state=42, loss="log_loss"
    )
    final_model.fit(X_train, y_train)

    # Evaluate once on test set
    print("\nEvaluating on held-out test set...")
    test_pred_proba = final_model.predict_proba(X_test)[:, 1]
    test_avg_precision = average_precision_score(y_test, test_pred_proba)

    print(f"Test Set Average Precision: {test_avg_precision:.3f}")

    return {
        "final_model": final_model,
        "cv_scores": cv_scores if run_cross_validation else None,
        "cv_mean": np.mean(cv_scores) if run_cross_validation else None,
        "cv_std": np.std(cv_scores) if run_cross_validation else None,
        "test_score": test_avg_precision,
        "model": model
    }


def create_param_grid(param_dict):
    """Convert a dictionary of parameter lists into a list of all possible combinations.

    Args:
        param_dict (dict): Dictionary where keys are parameter names and values are lists of parameter values

    Returns:
        list: List of dictionaries, each containing one combination of parameters
    """
    keys = param_dict.keys()
    values = param_dict.values()
    combinations = list(product(*values))
    return [dict(zip(keys, combo)) for combo in combinations]


def run_single_parameter_set(args):
    """Helper function to run a single parameter combination.

    Unpacks arguments and calls the training function.

    Args:
        args (tuple): Tuple containing (X, y, groups, params, param_id)

    Returns:
        dict: Results dictionary including parameters and scores
    """
    X, y, groups, params, param_id = args

    try:
        results = train_and_evaluate_final_model(X=X, y=y, groups=groups, **params)

        return {
            "param_id": param_id,
            "parameters": params,
            "cv_mean": results["cv_mean"],
            "cv_std": results["cv_std"],
            "test_score": results["test_score"],
            "status": "completed",
            "error": None,
        }
    except Exception as e:
        return {
            "param_id": param_id,
            "parameters": params,
            "cv_mean": None,
            "cv_std": None,
            "test_score": None,
            "status": "failed",
            "error": str(e),
        }


def run_parallel_grid_search(
    X, y, groups, param_grid, n_workers=None, results_dir="grid_search_results"
):
    """Run grid search in parallel using ProcessPoolExecutor.

    Args:
        X (array-like): Feature matrix
        y (array-like): Target vector
        groups (array-like): Groups for cross-validation
        param_grid (dict): Dictionary of parameters to search
        n_workers (int, optional): Number of parallel workers. Defaults to None (CPU count)
        results_dir (str, optional): Directory to save results. Defaults to 'grid_search_results'

    Returns:
        pd.DataFrame: DataFrame containing all results
    """
    # Create results directory if it doesn't exist
    os.makedirs(results_dir, exist_ok=True)

    # Generate all parameter combinations
    param_combinations = create_param_grid(param_grid)
    print(f"Total parameter combinations to test: {len(param_combinations)}")

    # Create timestamp for this run
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Prepare arguments for parallel execution
    args_list = [
        (X, y, groups, params, i) for i, params in enumerate(param_combinations)
    ]

    results = []

    # Run parallel execution
    with ProcessPoolExecutor(max_workers=n_workers) as executor:
        for result in executor.map(run_single_parameter_set, args_list):
            results.append(result)

            # Save intermediate results
            df = pd.DataFrame(results)
            df.to_csv(f"{results_dir}/grid_search_results_{timestamp}.csv", index=False)

            # Print progress
            completed = len([r for r in results if r["status"] == "completed"])
            failed = len([r for r in results if r["status"] == "failed"])
            print(
                f"\rProgress: {completed + failed}/{len(param_combinations)} "
                f"(Completed: {completed}, Failed: {failed})",
                end="",
            )

    print("\nGrid search completed!")

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Sort by CV mean score (descending)
    results_df = results_df.sort_values("cv_mean", ascending=False)

    # Save final results
    final_path = f"{results_dir}/grid_search_results_{timestamp}_final.csv"
    results_df.to_csv(final_path, index=False)
    print(f"\nFinal results saved to: {final_path}")

    return results_df



In [ ]:
features_list = [
            "eQtlColocClppMaximum",
            "pQtlColocClppMaximum",
            "sQtlColocClppMaximum",
            "eQtlColocH4Maximum",
            "pQtlColocH4Maximum",
            "sQtlColocH4Maximum",
            "eQtlColocClppMaximumNeighbourhood",
            "pQtlColocClppMaximumNeighbourhood",
            "sQtlColocClppMaximumNeighbourhood",
            "eQtlColocH4MaximumNeighbourhood",
            "pQtlColocH4MaximumNeighbourhood",
            "sQtlColocH4MaximumNeighbourhood",
            "distanceSentinelFootprint",
            "distanceSentinelFootprintNeighbourhood",
            "distanceFootprintMean",
            "distanceFootprintMeanNeighbourhood",
            "distanceTssMean",
            "distanceTssMeanNeighbourhood",
            "distanceSentinelTss",
            "distanceSentinelTssNeighbourhood",
            "vepMaximum",
            "vepMaximumNeighbourhood",
            "vepMean",
            "vepMeanNeighbourhood",
            "geneCount500kb",
            "proteinGeneCount500kb",
            "credibleSetConfidence",
        ]

In [ ]:
spark = SparkSession.builder.getOrCreate()
gs = spark.read.json("/Users/irenelopez/EBI/repos/gentropy/2506_traing_set.json")

print(gs.count())
print(gs.filter(f.size("diseaseIds") > 1).count())

In [ ]:
spark = SparkSession.builder.getOrCreate()

gs = spark.read.json("/Users/irenelopez/EBI/repos/gentropy/2506_traing_set.json") # gs://genetics-portal-dev-analysis/yt4/2506_release/training_set/2506_traing_set.json
fm = spark.read.parquet("/Users/irenelopez/EBI/repos/gentropy/l2g_feature_matrix") # gs://open-targets-pipeline-runs/szsz/25.06-testrun-4/intermediate/l2g_feature_matrix")

data_df = gs.join(fm, on=["geneId", "studyLocusId"], how="inner").coalesce(1).toPandas()


data_df["goldStandardSet"].value_counts()

In [ ]:
# Encode labels in `goldStandardSet` to a numeric value
label_encoder: dict[str, int] = {
            "negative": 0,
            "positive": 1,
        }
data_df["goldStandardSet"] = data_df["goldStandardSet"].map(label_encoder)

# Define sets
X = data_df[features_list].apply(pd.to_numeric).values
y = data_df["goldStandardSet"].apply(pd.to_numeric).values
# traits come now as arrays, I will convert it to a string column to create the groups
data_df["diseaseIds"] = [",".join(map(str, l)) for l in data_df["diseaseIds"]]
gene_groups = data_df["geneId"].astype(str)  # Group identifier has to be a single string


In [ ]:

param_grid = {
    "learning_rate": [0.01],
    "subsample": [0.5],
    "max_depth": [3],
    "ccp_alpha": [0.0],
    "min_samples_leaf": [1],
    "min_samples_split": [2]
}

results = run_parallel_grid_search(
    X=X,
    y=y,
    groups=gene_groups,
    param_grid=param_grid,
    n_workers=4  # Adjust based on your CPU cores
)

# Get top 5 parameter combinations
print("\nTop 5 parameter combinations:")
print(results.head())

In [ ]:
hyperparameters = {
    "ccp_alpha": 0.0,
    "learning_rate": 0.1,
    "max_depth": 3,
    "min_samples_leaf": 1,
    "min_samples_split": 5,
    "subsample": 0.7,
    "tol": 0.01,
}


result = train_and_evaluate_final_model(
    X,
    y,
    gene_groups,
    run_cross_validation=True,
    **hyperparameters,
)

In [ ]:
model = result["final_model"]

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

# Assuming 'model' is your trained GradientBoostingClassifier
# Visualize the first tree
plt.figure(figsize=(20, 10))
tree.plot_tree(model.estimators_[0][0],
               filled=True,
               rounded=True,
               proportion=True,
               fontsize=10)
plt.show()


In [ ]:
import skops.io as sio

sio.dump(model, "split_by_gene_model.skops")

## Evaluating predictions

In [ ]:
from gentropy.common.session import Session
from gentropy.dataset.l2g_feature_matrix import L2GFeatureMatrix
from gentropy.dataset.l2g_prediction import L2GPrediction

session = Session()

# credible_set = StudyLocus.from_parquet(
#     session, "/Users/irenelopez/EBI/repos/gentropy/credible_set"
# )
feature_matrix = L2GFeatureMatrix(
    _df=session.spark.read.parquet(
        "/Users/irenelopez/EBI/repos/gentropy/l2g_feature_matrix"
    ),
)


In [ ]:
predictions = L2GPrediction.from_credible_set(
                session,
                credible_set,
                feature_matrix,
                model_path="/Users/irenelopez/EBI/repos/gentropy/notebooks",
                features_list=features_list,
                hf_token=None,
                hf_model_version=None,
                download_from_hub=False,
            )

In [ ]:
new_pred = session.load_data("/Users/irenelopez/EBI/repos/gentropy/predictions_old_gs_hier")
pred = session.load_data("/Users/irenelopez/EBI/repos/gentropy/l2g_prediction").withColumnRenamed("score", "score_priv")

joined_df = new_pred.join(pred, ["studyLocusId", "geneId"], "inner").select("studyLocusId", "geneId", "score", "score_priv")
joined_df.count()

In [ ]:
import pandas as pd


def calculate_model_score(cv_precision_mean, cv_precision_std, test_precision, selectivity_2_count,
                         cv_weight=0.35, stability_weight=0.35, generalization_weight=0.1, selectivity_weight=0.4):
    """Calculate a composite score for model evaluation based on:
    1. CV precision performance
    2. CV precision stability (low variance)
    3. Generalization (CV vs Test precision difference)
    4. Selectivity (preference for ~20K in category 2)
    
    Parameters:
    -----------
    cv_precision_mean : float
        Mean cross-validation precision
    cv_precision_std : float
        Standard deviation of cross-validation precision
    test_precision : float
        Test set precision
    selectivity_2_count : int
        Count of category 2 in selectivity (target ~20,000)
    cv_weight : float
        Weight for CV precision component (default 0.2)
    stability_weight : float
        Weight for stability component (default 0.2)
    generalization_weight : float
        Weight for generalization component (default 0.2)
    selectivity_weight : float
        Weight for selectivity component (default 0.4)
    
    Returns:
    --------
    dict : Dictionary containing individual scores and composite score
    """
    # 1. CV Precision Score (0-1, higher is better)
    cv_score = cv_precision_mean

    # 2. Stability Score (0-1, higher is better - penalize high variance)
    # Use exponential decay to heavily penalize high variance
    stability_score = np.exp(-10 * cv_precision_std)

    # 3. Generalization Score (0-1, higher is better - penalize large CV-Test gap)
    precision_gap = abs(cv_precision_mean - test_precision)
    generalization_score = np.exp(-20 * precision_gap)

    # 4. Selectivity Score (0-1, higher is better - target ~20K for category 2)
    target_selectivity = 20000
    selectivity_ratio = min(selectivity_2_count, target_selectivity) / target_selectivity
    # Bonus if very close to target, penalty if too far
    if selectivity_2_count > target_selectivity:
        selectivity_score = selectivity_ratio * np.exp(-0.5 * (selectivity_2_count - target_selectivity) / target_selectivity)
    else:
        selectivity_score = selectivity_ratio

    # Composite Score (weighted average)
    composite_score = (
        cv_weight * cv_score +
        stability_weight * stability_score +
        generalization_weight * generalization_score +
        selectivity_weight * selectivity_score
    )

    return {
        "cv_score": cv_score,
        "stability_score": stability_score,
        "generalization_score": generalization_score,
        "selectivity_score": selectivity_score,
        "composite_score": composite_score
    }

def evaluate_models_from_table():
    """Evaluate the models from your table
    """
    # Your data (extracted from the table)
    models_data = [
        {"cv_mean": 0.832, "cv_std": 0.103, "test_precision": 0.835, "selectivity_2": 21728},
        {"cv_mean": 0.704, "cv_std": 0.124, "test_precision": 0.74, "selectivity_2": None},  # Missing data
        {"cv_mean": 0.6997, "cv_std": 0.0934, "test_precision": 0.7725, "selectivity_2": 42000},
        {"cv_mean": 0.8206, "cv_std": 0.0589, "test_precision": 0.8459, "selectivity_2": 70484},
        {"cv_mean": 0.875, "cv_std": 0.034, "test_precision": 0.898, "selectivity_2": 110000},
        {"cv_mean": 0.844, "cv_std": 0.071, "test_precision": 0.778, "selectivity_2": 90000}
    ]

    results = []
    for i, model in enumerate(models_data):
        if model["selectivity_2"] is not None:  # Skip model with missing data
            scores = calculate_model_score(
                model["cv_mean"],
                model["cv_std"],
                model["test_precision"],
                model["selectivity_2"]
            )
            scores["model_id"] = i + 1
            results.append(scores)

    # Convert to DataFrame for easy viewing
    df_results = pd.DataFrame(results)
    df_results = df_results[["model_id", "cv_score", "stability_score", "generalization_score",
                            "selectivity_score", "composite_score"]]
    df_results = df_results.round(4)

    return df_results

In [ ]:
results = evaluate_models_from_table()
print("Model Evaluation Results (sorted by composite score):")
print("=" * 80)
print(results.to_string(index=False))

print("\n" + "=" * 80)
print("Score Interpretation:")
print("- CV Score: Direct CV precision (higher = better)")
print("- Stability Score: Exponential penalty for high variance (higher = better)")
print("- Generalization Score: Exponential penalty for CV-Test gap (higher = better)")
print("- Selectivity Score: Preference for ~20K category 2 hits (higher = better)")
print("- Composite Score: Weighted combination (higher = better)")

# Show the best model
best_model = results.iloc[0]
print(f"\nBest Model: Model {int(best_model['model_id'])} with composite score: {best_model['composite_score']:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Collect the data into a Pandas DataFrame
pandas_df = joined_df.select("score", "score_priv").toPandas()

# Plot the data
plt.figure(figsize=(10, 6))
plt.scatter(pandas_df["score"], pandas_df["score_priv"], alpha=0.005)
plt.title("Scatter Plot of Score vs. Priv Score")
plt.xlabel("Score")
plt.ylabel("Priv Score")
plt.grid(True)
plt.show()

In [ ]:
# Calculate the correlation between score and priv_score
correlation = joined_df.corr("score", "score_priv")

# Show the correlation
print(f"Correlation between score and priv_score: {correlation}")

In [ ]:
from pyspark.sql import functions as f

In [ ]:

(
    new_pred.filter(f.col("score") >= 0.5)
    .groupBy("studyLocusId")
    .agg(f.collect_set("geneId").alias("geneIds"))
    .withColumn(
        "hits",
        f.when(f.size("geneIds") == 1, f.lit("1"))
        .when((f.size("geneIds") == 2), f.lit("2"))
        .otherwise(f.lit(">2")),
    )
    .groupBy("hits")
    .count()
    .orderBy(f.col("count").desc())
    .show()
)

In [ ]:
feature_matrix._df.filter(
    (f.col("distanceTssMean") > 1) | (f.col("distanceSentinelTss") > 1)
).select("distanceTssMean", "distanceSentinelTss").describe().show()

In [ ]:
import pandas as pd

pd.read_json("/Users/irenelopez/EBI/repos/gentropy/cttv008_2024-08-09.json.gz", lines=True).query("studyId == 'NCT05329194'").head()

## More intelligent splitting

In [ ]:
data_df["goldStandardSet"].value_counts()

In [ ]:
data_df.iloc[0]

In [ ]:
positive_X = data_df[data_df["goldStandardSet"] == 1][features_list].reset_index(drop=True)
negative_X = data_df[data_df["goldStandardSet"] == 0][features_list].reset_index(drop=True)
positive_y = data_df[data_df["goldStandardSet"] == 1]["goldStandardSet"].reset_index(drop=True)
negative_y = data_df[data_df["goldStandardSet"] == 0]["goldStandardSet"].reset_index(drop=True)
positive_groups = data_df[data_df["goldStandardSet"] == 1]["geneId"].reset_index(drop=True)

print(f"Positive X: {positive_X.shape[0]}")
print(f"Negative X: {negative_X.shape[0]}")
print(f"Positive y: {positive_y.shape[0]}")
print(f"Negative y: {negative_y.shape[0]}")


In [ ]:
positives_train_test_split = GroupShuffleSplit(
        n_splits=1, test_size=0.2, random_state=42
    )
for train_idx, test_idx in positives_train_test_split.split(positive_X, positive_y, positive_groups):
    X_train_pos, X_test_pos = positive_X[train_idx], positive_X[test_idx]
    y_train_pos, y_test_pos = positive_y[train_idx], positive_y[test_idx]
    groups_train_pos = positive_groups[train_idx]

In [ ]:
positives = data_df[data_df["goldStandardSet"] == 1].copy()
negatives = data_df[data_df["goldStandardSet"] == 0].copy()

print(f"Total samples: {len(data_df)}")
print(f"Positives: {len(positives)}")
print(f"Negatives: {len(negatives)}")
print(f"Unique genes in positives: {positives['geneId'].nunique()}")
print(f"Unique studyLocusIds in positives: {positives['studyLocusId'].nunique()}")

In [ ]:
# Step 1: Group positives by geneId and split genes between train/test
gene_groups = positives.groupby("geneId").size().reset_index(name="count")
gene_groups = gene_groups.sort_values("count", ascending=False)
gene_groups.head()

In [ ]:
from sklearn.model_selection import train_test_split

genes_train, genes_test = train_test_split(
        gene_groups["geneId"].tolist(),
        test_size=0.15,
        random_state=42,
        shuffle=True
    )
print("\nGene-level split:")
print(f"Genes in train: {len(genes_train)}")
print(f"Genes in test: {len(genes_test)}")

In [ ]:
# Step 2: Split by studyLocusId within each gene group
train_study_loci = set()
test_study_loci = set()

# Get studyLocusIds for train genes
train_gene_positives = positives[positives["geneId"].isin(genes_train)]
train_study_loci.update(train_gene_positives["studyLocusId"].unique())

# Get studyLocusIds for test genes
test_gene_positives = positives[positives["geneId"].isin(genes_test)]
test_study_loci.update(test_gene_positives["studyLocusId"].unique())

assert len(train_study_loci.intersection(test_study_loci)) == 0, "Overlapping studyLocusIds found in both gene groups"

# Final positive splits
train_positives = positives[positives["studyLocusId"].isin(train_study_loci)]
test_positives = positives[positives["studyLocusId"].isin(test_study_loci)]

print("\nStudyLocusId-level split:")
print(f"StudyLocusIds in train: {len(train_study_loci)}")
print(f"StudyLocusIds in test: {len(test_study_loci)}")
print(f"Positive samples in train: {len(train_positives)}")
print(f"Positive samples in test: {len(test_positives)}")


In [ ]:
# Step 3: Augment with negatives based on studyLocusId
train_negatives = negatives[negatives["studyLocusId"].isin(train_study_loci)]
test_negatives = negatives[negatives["studyLocusId"].isin(test_study_loci)]

train_df = pd.concat([train_positives, train_negatives], ignore_index=True)
test_df = pd.concat([test_positives, test_negatives], ignore_index=True)



In [ ]:
# Final validation
train_genes = set(train_df["geneId"].unique())
test_genes = set(test_df["geneId"].unique())
train_loci = set(train_df["studyLocusId"].unique())
test_loci = set(test_df["studyLocusId"].unique())

gene_overlap = train_genes.intersection(test_genes)
loci_overlap = train_loci.intersection(test_loci)

print("\nFinal split statistics:")
print(f"Train set: {len(train_df)} samples ({train_df['goldStandardSet'].sum()} positives)")
print(f"Test set: {len(test_df)} samples ({test_df['goldStandardSet'].sum()} positives)")
print(f"Gene overlap between splits (expected): {len(gene_overlap)}")
print(f"StudyLocusId overlap between splits: {len(loci_overlap)}")

if loci_overlap:
    print("Data leakage detected! Overlapping studyLocusIds between splits.")
else:
    print("✓ No data leakage detected!")

In [ ]:
def hierarchical_split(data_df: pd.DataFrame,
                      test_size: float = 0.15,
                      random_state: int = 42,
                      verbose: bool = True) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Implements hierarchical splitting strategy to prevent data leakage.
    
    Strategy:
    1. Split ALL genes (from both positives and negatives) into train/test groups
    2. Assign studyLocusIds based on positive gene assignments
    3. Filter all samples (positives and negatives) based on gene assignments
    4. Resolve any studyLocusId conflicts to prevent leakage
    
    Args:
        data_df (pd.DataFrame): Input dataframe with goldStandardSet column (1=positive, 0=negative)
        test_size (float): Proportion of data for test set. Defaults to 0.15.
        random_state (int): Random seed for reproducibility. Defaults to 42.
        verbose (bool): Print splitting statistics. Defaults to True.
        
    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: Training and test dataframes
    """
    # Separate positives and negatives
    positives = data_df[data_df["goldStandardSet"] == 1].copy()
    negatives = data_df[data_df["goldStandardSet"] == 0].copy()

    if verbose:
        print(f"Total samples: {len(data_df)}")
        print(f"Positives: {len(positives)}")
        print(f"Negatives: {len(negatives)}")
        print(f"Unique genes in positives: {positives['geneId'].nunique()}")
        print(f"Unique genes in negatives: {negatives['geneId'].nunique()}")
        print(f"Unique genes overall: {data_df['geneId'].nunique()}")
        print(f"Unique studyLocusIds in positives: {positives['studyLocusId'].nunique()}")

    # Step 1: Group positives by geneId and split genes between train/test
    gene_groups = positives.groupby("geneId").size().reset_index(name="count")
    gene_groups = gene_groups.sort_values("count", ascending=False)

    # Split genes maintaining approximate test_size proportion
    genes_train, genes_test = train_test_split(
        gene_groups["geneId"].tolist(),
        test_size=test_size,
        random_state=random_state,
        shuffle=True
    )

    if verbose:
        print("\nGene-level split:")
        print(f"Genes in train: {len(genes_train)}")
        print(f"Genes in test: {len(genes_test)}")

    # Step 2: Split by studyLocusId within each gene group
    train_study_loci = set()
    test_study_loci = set()

    # Get studyLocusIds for train genes
    train_gene_positives = positives[positives["geneId"].isin(genes_train)]
    train_study_loci.update(train_gene_positives["studyLocusId"].unique())

    # Get studyLocusIds for test genes
    test_gene_positives = positives[positives["geneId"].isin(genes_test)]
    test_study_loci.update(test_gene_positives["studyLocusId"].unique())

    # Check for overlap in studyLocusIds between train and test
    assert len(train_study_loci.intersection(test_study_loci)) == 0, "Overlapping studyLocusIds found in both gene groups"

    # Final positive splits
    train_positives = positives[positives["studyLocusId"].isin(train_study_loci)]
    test_positives = positives[positives["studyLocusId"].isin(test_study_loci)]

    if verbose:
        print("\nStudyLocusId-level split:")
        print(f"StudyLocusIds in train: {len(train_study_loci)}")
        print(f"StudyLocusIds in test: {len(test_study_loci)}")
        print(f"Positive samples in train: {len(train_positives)}")
        print(f"Positive samples in test: {len(test_positives)}")

    # Step 3: Augment with negatives based on studyLocusId
    train_negatives = negatives[negatives["studyLocusId"].isin(train_study_loci)]
    test_negatives = negatives[negatives["studyLocusId"].isin(test_study_loci)]

    # Combine positives and negatives for final splits
    train_df = pd.concat([train_positives, train_negatives], ignore_index=True)
    test_df = pd.concat([test_positives, test_negatives], ignore_index=True)

    # Final validation
    train_genes = set(train_df["geneId"].unique())
    test_genes = set(test_df["geneId"].unique())
    train_loci = set(train_df["studyLocusId"].unique())
    test_loci = set(test_df["studyLocusId"].unique())

    gene_overlap = train_genes.intersection(test_genes)
    loci_overlap = train_loci.intersection(test_loci)

    if verbose:
        print("\nFinal split statistics:")
        print(f"Train set: {len(train_df)} samples ({train_df['goldStandardSet'].sum()} positives)")
        print(f"Test set: {len(test_df)} samples ({test_df['goldStandardSet'].sum()} positives)")
        print(f"Gene overlap between splits (expected): {len(gene_overlap)}")
        print(f"StudyLocusId overlap between splits (not expected): {len(loci_overlap)}")

        if loci_overlap:
            print("Data leakage detected! Overlapping studyLocusIds between splits.")
        else:
            print("✓ No data leakage detected!")

    return train_df, test_df


In [ ]:
def analyze_split_quality(train_df: pd.DataFrame, test_df: pd.DataFrame) -> dict:
    """Analyze the quality of the hierarchical split.
    
    Parameters:
    -----------
    train_df, test_df : pd.DataFrame
        Training and test dataframes
        
    Returns:
    --------
    Dict
        Dictionary containing split quality metrics
    """
    analysis = {}

    # Basic statistics
    analysis["train_size"] = len(train_df)
    analysis["test_size"] = len(test_df)
    analysis["train_positives"] = train_df["goldStandardSet"].sum()
    analysis["test_positives"] = test_df["goldStandardSet"].sum()
    analysis["train_pos_ratio"] = analysis["train_positives"] / len(train_df)
    analysis["test_pos_ratio"] = analysis["test_positives"] / len(test_df)

    # Gene-level analysis
    train_genes = set(train_df["geneId"].unique())
    test_genes = set(test_df["geneId"].unique())
    analysis["unique_genes_train"] = len(train_genes)
    analysis["unique_genes_test"] = len(test_genes)
    analysis["gene_overlap"] = len(train_genes.intersection(test_genes))

    # StudyLocusId-level analysis
    train_loci = set(train_df["studyLocusId"].unique())
    test_loci = set(test_df["studyLocusId"].unique())
    analysis["unique_loci_train"] = len(train_loci)
    analysis["unique_loci_test"] = len(test_loci)
    analysis["loci_overlap"] = len(train_loci.intersection(test_loci))

    # Data leakage check
    analysis["no_leakage"] = (analysis["gene_overlap"] == 0 and analysis["loci_overlap"] == 0)

    return analysis


analysis = analyze_split_quality(train_df, test_df)
print("\nSplit Quality Analysis:")
for key, value in analysis.items():
    print(f"{key}: {value}")

In [ ]:
train_df, test_df = hierarchical_split(data_df, test_size=0.2, random_state=42)

In [ ]:

def train_and_evaluate_final_model(
    X,
    y,
    groups,
    test_size=0.2,
    n_splits=5,
    run_cross_validation=True,
    **hyperparameters,
):
    """Implementation of the final training strategy."""
    # Create held-out test set separating EFO/Gene pairs between train/test
    train_test_split = GroupShuffleSplit(
        n_splits=1, test_size=test_size, random_state=42
    )
    for train_idx, test_idx in train_test_split.split(X, y, groups):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        groups_train = groups[train_idx]

    # Cross-validation
    if run_cross_validation:
        cv_scores = []
        gkf = GroupKFold(n_splits=n_splits)
        print("Performing cross-validation...")
        for fold, (train_idx, val_idx) in enumerate(
            gkf.split(X_train, y_train, groups_train)
        ):
            # Split data
            X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
            y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

            model = GradientBoostingClassifier(
                **hyperparameters, random_state=42, loss="log_loss"
            )
            model.fit(X_fold_train, y_fold_train)
            y_pred_proba = model.predict_proba(X_fold_val)[:, 1]
            avg_precision = average_precision_score(y_fold_val, y_pred_proba)
            cv_scores.append(avg_precision)

            print(f"Fold {fold + 1}: Average Precision = {avg_precision:.3f}")

        print(
            f"\nCross-validation Average Precision: {np.mean(cv_scores):.3f} (+/- {np.std(cv_scores) * 2:.3f})"
        )

    # Train final model on full training set
    print("\nTraining final model on full training set...")
    final_model = GradientBoostingClassifier(
        **hyperparameters, random_state=42, loss="log_loss"
    )
    final_model.fit(X_train, y_train)

    # Evaluate once on test set
    print("\nEvaluating on held-out test set...")
    test_pred_proba = final_model.predict_proba(X_test)[:, 1]
    test_avg_precision = average_precision_score(y_test, test_pred_proba)

    print(f"Test Set Average Precision: {test_avg_precision:.3f}")

    return {
        "final_model": final_model,
        "cv_scores": cv_scores if run_cross_validation else None,
        "cv_mean": np.mean(cv_scores) if run_cross_validation else None,
        "cv_std": np.std(cv_scores) if run_cross_validation else None,
        "test_score": test_avg_precision,
        "model": model
    }



In [ ]:

def hierarchical_cross_validation_split(train_df, n_splits=5, random_state=42):
    """Creates cross-validation splits that maintain hierarchical gene/studyLocusId constraints.
    
    Parameters:
    -----------
    train_df : pd.DataFrame
        Training dataframe
    n_splits : int, default=5
        Number of CV folds
    random_state : int, default=42
        Random seed
        
    Yields:
    -------
    tuple
        (train_indices, val_indices) for each fold
    """
    # Get unique genes from positives only (maintaining hierarchy)
    positives = train_df[train_df["goldStandardSet"] == 1]
    unique_genes = positives["geneId"].unique()

    # Split genes into folds
    np.random.seed(random_state)
    np.random.shuffle(unique_genes)
    gene_folds = np.array_split(unique_genes, n_splits)

    for fold_idx in range(n_splits):
        # Validation genes for this fold
        val_genes = set(gene_folds[fold_idx])

        # Training genes for this fold (all others)
        train_genes = set()
        for other_fold_idx in range(n_splits):
            if other_fold_idx != fold_idx:
                train_genes.update(gene_folds[other_fold_idx])

        # Get studyLocusIds for each gene group (from positives only)
        train_gene_positives = positives[positives["geneId"].isin(train_genes)]
        val_gene_positives = positives[positives["geneId"].isin(val_genes)]

        train_study_loci = set(train_gene_positives["studyLocusId"].unique())
        val_study_loci = set(val_gene_positives["studyLocusId"].unique())

        # Resolve conflicts (assign overlapping studyLocusIds to train)
        overlapping_loci = train_study_loci.intersection(val_study_loci)
        if overlapping_loci:
            val_study_loci = val_study_loci - overlapping_loci
            train_study_loci = train_study_loci.union(overlapping_loci)

        # Get final indices for this fold
        train_mask = (
            (train_df["geneId"].isin(train_genes)) &
            (train_df["studyLocusId"].isin(train_study_loci))
        )
        val_mask = (
            (train_df["geneId"].isin(val_genes)) &
            (train_df["studyLocusId"].isin(val_study_loci))
        )

        train_indices = train_df.index[train_mask].tolist()
        val_indices = train_df.index[val_mask].tolist()

        yield train_indices, val_indices


def train_and_evaluate_with_presplit(
    train_df,
    test_df,
    feature_columns,
    n_splits=5,
    run_cross_validation=True,
    target_column="goldStandardSet",
    **hyperparameters,
):
    """Implementation of training strategy with hierarchical cross-validation.
    
    Parameters:
    -----------
    train_df : pd.DataFrame
        Training dataframe from hierarchical split
    test_df : pd.DataFrame
        Test dataframe from hierarchical split
    feature_columns : list
        List of feature column names
    n_splits : int, default=5
        Number of cross-validation folds
    run_cross_validation : bool, default=True
        Whether to perform cross-validation
    target_column : str, default='goldStandardSet'
        Name of target column
    **hyperparameters : dict
        Hyperparameters for GradientBoostingClassifier
        
    Returns:
    --------
    dict
        Dictionary containing model and evaluation results
    """
    # Prepare data
    X_train = train_df[feature_columns].values
    y_train = train_df[target_column].values
    X_test = test_df[feature_columns].values
    y_test = test_df[target_column].values

    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")
    print(f"Training positives: {y_train.sum()}/{len(y_train)} ({y_train.mean():.3f})")
    print(f"Test positives: {y_test.sum()}/{len(y_test)} ({y_test.mean():.3f})")

    # Cross-validation with hierarchical splits
    cv_scores = []
    if run_cross_validation:
        print(f"\nPerforming hierarchical cross-validation ({n_splits} folds)...")

        for fold, (train_indices, val_indices) in enumerate(
            hierarchical_cross_validation_split(train_df, n_splits=n_splits)
        ):
            if len(val_indices) == 0:
                print(f"Fold {fold + 1}: Skipped (no validation samples)")
                continue

            # Convert DataFrame indices to array indices
            train_array_idx = [train_df.index.get_loc(idx) for idx in train_indices]
            val_array_idx = [train_df.index.get_loc(idx) for idx in val_indices]

            # Split data for this fold
            X_fold_train, X_fold_val = X_train[train_array_idx], X_train[val_array_idx]
            y_fold_train, y_fold_val = y_train[train_array_idx], y_train[val_array_idx]

            # Validate no gene/studyLocusId overlap
            fold_train_genes = set(train_df.iloc[train_array_idx]["geneId"])
            fold_val_genes = set(train_df.iloc[val_array_idx]["geneId"])
            fold_train_loci = set(train_df.iloc[train_array_idx]["studyLocusId"])
            fold_val_loci = set(train_df.iloc[val_array_idx]["studyLocusId"])

            gene_overlap = len(fold_train_genes.intersection(fold_val_genes))
            loci_overlap = len(fold_train_loci.intersection(fold_val_loci))

            # Train model on fold
            model = GradientBoostingClassifier(
                **hyperparameters, random_state=42, loss="log_loss"
            )
            model.fit(X_fold_train, y_fold_train)

            # Evaluate on validation fold
            y_pred_proba = model.predict_proba(X_fold_val)[:, 1]
            avg_precision = average_precision_score(y_fold_val, y_pred_proba)
            cv_scores.append(avg_precision)

            print(f"Fold {fold + 1}: AP = {avg_precision:.3f} | "
                  f"Train: {len(X_fold_train)} samples | Val: {len(X_fold_val)} samples | "
                  f"Gene overlap: {gene_overlap} | Loci overlap: {loci_overlap}")

        if cv_scores:
            print(
                f"\nCross-validation Average Precision: {np.mean(cv_scores):.3f} (+/- {np.std(cv_scores) * 2:.3f})"
            )

    # Train final model on full training set
    print("\nTraining final model on full training set...")
    final_model = GradientBoostingClassifier(
        **hyperparameters, random_state=42, loss="log_loss"
    )
    final_model.fit(X_train, y_train)

    # Evaluate on held-out test set
    print("\nEvaluating on held-out test set...")
    test_pred_proba = final_model.predict_proba(X_test)[:, 1]
    test_avg_precision = average_precision_score(y_test, test_pred_proba)

    print(f"Test Set Average Precision: {test_avg_precision:.3f}")

    return {
        "final_model": final_model,
        "cv_scores": cv_scores if run_cross_validation else None,
        "cv_mean": np.mean(cv_scores) if cv_scores else None,
        "cv_std": np.std(cv_scores) if cv_scores else None,
        "test_score": test_avg_precision,
    }


def prepare_data_for_training(train_df, test_df, feature_columns, target_column="goldStandardSet", group_column="studyLocusId"):
    """Helper function to prepare data from hierarchical split for training.
    
    Parameters:
    -----------
    train_df : pd.DataFrame
        Training dataframe from hierarchical_split
    test_df : pd.DataFrame
        Test dataframe from hierarchical_split
    feature_columns : list
        List of column names to use as features
    target_column : str, default='goldStandardSet'
        Name of target column
    group_column : str, default='studyLocusId'
        Name of group column for cross-validation
        
    Returns:
    --------
    tuple
        (X_train, y_train, X_test, y_test, groups_train)
    """
    # Prepare training data
    X_train = train_df[feature_columns].values
    y_train = train_df[target_column].values
    groups_train = train_df[group_column].values

    # Prepare test data
    X_test = test_df[feature_columns].values
    y_test = test_df[target_column].values

    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")
    print(f"Training positives: {y_train.sum()}/{len(y_train)} ({y_train.mean():.3f})")
    print(f"Test positives: {y_test.sum()}/{len(y_test)} ({y_test.mean():.3f})")
    print(f"Unique groups in training: {len(np.unique(groups_train))}")

    return X_train, y_train, X_test, y_test, groups_train


# Example usage workflow:
def complete_training_workflow(data_df, feature_columns, hyperparameters, test_size=0.2, cv_folds=5):
    """Complete workflow: hierarchical split -> train and evaluate with hierarchical CV
    
    Parameters:
    -----------
    data_df : pd.DataFrame
        Full dataset
    feature_columns : list
        List of feature column names
    hyperparameters : dict
        Model hyperparameters
    test_size : float, default=0.2
        Test set proportion
    cv_folds : int, default=5
        Number of CV folds
        
    Returns:
    --------
    tuple
        (results_dict, train_df, test_df)
    """
    # Step 1: Hierarchical split
    print("=== Performing Hierarchical Split ===")
    train_df, test_df = hierarchical_split(data_df, test_size=test_size, random_state=42)

    # Step 2: Train and evaluate with hierarchical CV
    print("\n=== Training and Evaluating Model with Hierarchical CV ===")
    results = train_and_evaluate_with_presplit(
        train_df, test_df, feature_columns,
        n_splits=cv_folds,
        **hyperparameters
    )

    return results, train_df, test_df

hyperparameters = {
    "ccp_alpha": 0.0,
    "learning_rate": 0.1,
    "max_depth": 3,
    "min_samples_leaf": 1,
    "min_samples_split": 5,
    "subsample": 0.7,
    "tol": 0.01,
}

results, train_df, test_df = complete_training_workflow(
    data_df,
    features_list,
    hyperparameters,
    test_size=0.15,
    cv_folds=5
)

print(results)

"""
# OR use step by step:
# 1. Split data hierarchically
train_df, test_df = hierarchical_split(data_df, test_size=0.2)

# 2. Train and evaluate with hierarchical CV
results = train_and_evaluate_with_presplit(
    train_df, test_df, feature_columns,
    n_splits=5,
    **hyperparameters
)
"""

In [ ]:
import pandas as pd


def train_and_evaluate_final_model(
    data_df,  # Changed from separate X, y, groups to full dataframe
    feature_columns,  # List of feature column names
    target_column="goldStandardSet",  # Target column name
    test_size=0.2,
    n_splits=5,
    run_cross_validation=True,
    random_state=42,
    verbose=True,
    **hyperparameters,
):
    """Implementation of the final training strategy using hierarchical splitting.
    
    Parameters:
    -----------
    data_df : pd.DataFrame
        Full dataframe containing features, target, geneId, and studyLocusId
    feature_columns : list
        List of column names to use as features
    target_column : str, default='goldStandardSet'
        Name of target column
    test_size : float, default=0.2
        Proportion of data for test set
    n_splits : int, default=5
        Number of cross-validation folds
    run_cross_validation : bool, default=True
        Whether to perform cross-validation
    random_state : int, default=42
        Random seed for reproducibility
    verbose : bool, default=True
        Print detailed information
    **hyperparameters : dict
        Additional hyperparameters for GradientBoostingClassifier
        
    Returns:
    --------
    dict
        Dictionary containing model, scores, and evaluation metrics
    """
    print("=== Hierarchical Train/Test Split ===")
    # Create held-out test set using hierarchical splitting
    train_df, test_df = hierarchical_split(
        data_df,
        test_size=test_size,
        random_state=random_state,
        verbose=verbose
    )

    # Prepare train and test data
    X_train = train_df[feature_columns].values
    y_train = train_df[target_column].values
    X_test = test_df[feature_columns].values
    y_test = test_df[target_column].values

    if verbose:
        print(f"\nTraining set: {len(X_train)} samples, {y_train.sum()} positives")
        print(f"Test set: {len(X_test)} samples, {y_test.sum()} positives")

    # Cross-validation using hierarchical splitting
    cv_scores = []
    if run_cross_validation:
        print(f"\n=== {n_splits}-Fold Cross-Validation with Hierarchical Splitting ===")

        # Create CV folds using hierarchical splitting
        cv_train_dfs = []
        cv_val_dfs = []

        for fold in range(n_splits):
            # Use different random seeds for each fold
            fold_seed = random_state + fold

            # Split training data hierarchically for this fold
            cv_train, cv_val = hierarchical_split(
                train_df,
                test_size=1.0/n_splits,  # Approximate validation size
                random_state=fold_seed,
                verbose=False  # Reduce verbosity for CV folds
            )

            cv_train_dfs.append(cv_train)
            cv_val_dfs.append(cv_val)

        # Train and evaluate each fold
        for fold in range(n_splits):
            if verbose:
                print(f"\nFold {fold + 1}/{n_splits}:")

            # Get fold data
            fold_train_df = cv_train_dfs[fold]
            fold_val_df = cv_val_dfs[fold]

            X_fold_train = fold_train_df[feature_columns].values
            y_fold_train = fold_train_df[target_column].values
            X_fold_val = fold_val_df[feature_columns].values
            y_fold_val = fold_val_df[target_column].values

            if verbose:
                print(f"  Train: {len(X_fold_train)} samples, {y_fold_train.sum()} positives")
                print(f"  Val: {len(X_fold_val)} samples, {y_fold_val.sum()} positives")

                # Check for data leakage in this fold
                train_genes = set(fold_train_df["geneId"].unique())
                val_genes = set(fold_val_df["geneId"].unique())
                train_loci = set(fold_train_df["studyLocusId"].unique())
                val_loci = set(fold_val_df["studyLocusId"].unique())

                gene_overlap = len(train_genes.intersection(val_genes))
                loci_overlap = len(train_loci.intersection(val_loci))

                if gene_overlap > 0 or loci_overlap > 0:
                    print(f"  Warning: Fold {fold + 1} has leakage - Gene overlap: {gene_overlap}, Loci overlap: {loci_overlap}")

            # Train model for this fold
            fold_model = GradientBoostingClassifier(
                **hyperparameters,
                random_state=random_state,
                loss="log_loss"
            )

            fold_model.fit(X_fold_train, y_fold_train)

            # Predict and evaluate
            y_pred_proba = fold_model.predict_proba(X_fold_val)[:, 1]
            avg_precision = average_precision_score(y_fold_val, y_pred_proba)
            cv_scores.append(avg_precision)

            if verbose:
                print(f"  Average Precision: {avg_precision:.4f}")

        if verbose:
            print("\nCross-validation Results:")
            print(f"  Mean Average Precision: {np.mean(cv_scores):.4f}")
            print(f"  Std Average Precision: {np.std(cv_scores):.4f}")
            print(f"  95% CI: {np.mean(cv_scores):.4f} ± {np.std(cv_scores) * 1.96:.4f}")

    # Train final model on full training set
    print("\n=== Training Final Model ===")
    final_model = GradientBoostingClassifier(
        **hyperparameters,
        random_state=random_state,
        loss="log_loss"
    )

    if verbose:
        print(f"Training on {len(X_train)} samples with {len(feature_columns)} features...")

    final_model.fit(X_train, y_train)

    # Evaluate on held-out test set
    print("\n=== Final Test Set Evaluation ===")
    test_pred_proba = final_model.predict_proba(X_test)[:, 1]
    test_avg_precision = average_precision_score(y_test, test_pred_proba)

    if verbose:
        print(f"Test Set Average Precision: {test_avg_precision:.4f}")

    # Return comprehensive results
    results = {
        "final_model": final_model,
        "train_df": train_df,
        "test_df": test_df,
        "X_train": X_train,
        "y_train": y_train,
        "X_test": X_test,
        "y_test": y_test,
        "test_predictions": test_pred_proba,
        "cv_scores": cv_scores if run_cross_validation else None,
        "cv_mean": np.mean(cv_scores) if run_cross_validation and cv_scores else None,
        "cv_std": np.std(cv_scores) if run_cross_validation and cv_scores else None,
        "test_score": test_avg_precision,
        "feature_columns": feature_columns,
        "hyperparameters": hyperparameters
    }

    return results


def simple_train_and_evaluate(
    data_df,
    feature_columns,
    target_column="goldStandardSet",
    test_size=0.2,
    random_state=42,
    **hyperparameters
):
    """Simplified version for quick model training and evaluation.
    
    Parameters:
    -----------
    data_df : pd.DataFrame
        Full dataframe containing features, target, geneId, and studyLocusId
    feature_columns : list
        List of column names to use as features
    target_column : str, default='goldStandardSet'
        Name of target column
    test_size : float, default=0.2
        Proportion of data for test set
    random_state : int, default=42
        Random seed for reproducibility
    **hyperparameters : dict
        Additional hyperparameters for GradientBoostingClassifier
        
    Returns:
    --------
    dict
        Dictionary containing model and test score
    """
    return train_and_evaluate_final_model(
        data_df=data_df,
        feature_columns=feature_columns,
        target_column=target_column,
        test_size=test_size,
        n_splits=0,  # Skip cross-validation
        run_cross_validation=False,
        random_state=random_state,
        verbose=True,
        **hyperparameters
    )


results = train_and_evaluate_final_model(
    data_df=data_df,
    feature_columns=features_list,
    **hyperparameters
)

In [ ]:
model = results["final_model"]
model

In [ ]:
import skops.io as sio

sio.dump(model, "classifier.skops")